## Sales Operations - Lost invalid deals


In this code, we will access Pipedrive through the API and capture all Organizations, all Deals, and constantly update the Deals.

Next, we will construct the DealValidation table, which will link Organizations with Deals. We will proceed to associate the company's Website with its respective deal.

We will apply a Website Validation function to check the website of each Organization and create a list containing all the websites that are not valid. We will then apply a "lost" status to their respective deals.

This way, any Deal with an invalid website will be removed from the sales funnel.

### 1 - Creating Functions and Tables

In [40]:
#Importing Packagens
import requests
import json
import pandas as pd
import os
from datetime import date
from urllib.parse import urlparse
import tldextract
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import time
from datetime import datetime, timedelta
import sqlite3


In [41]:
#--- API Pipedrive:
# https://developers.pipedrive.com/docs/api/v1

#Accessing Keys
your_token = "" #Insert your token here
token = f'?api_token={your_token}'
pipedriveURL = 'https://snovio-5a30ce.pipedrive.com/api/v1/'

In [42]:
#--- Creating Functions

# Function to transform the URL
def make_https(url):
    if url and isinstance(url, str):
        # Remove "http://", "https://", and "www."
        url = url.replace("http://", "").replace("https://", "").replace("www.", "")
        # Extract only the domain and site name
        parsed_url = urlparse(url)
        return f"https://www.{parsed_url.netloc}{parsed_url.path}"
    else:
        return url

# Creating a session for persistent connections
session = requests.Session()

# Creating a get_status function that uses the session
def get_status(url):
    if url and isinstance(url, str):
        try:
            # Use the session for persistent connections
            response = requests.head(url, timeout=10, allow_redirects=True)
            
            # Check the status code
            if response.status_code == 200 or response.status_code == 403:
                return 'valid'
            elif response.status_code == 404:
                return 'invalid'
            else:
                return 'other HTML response'
        except requests.Timeout:
            return 'timed out'
        except Exception as e:
            # Handle exceptions, if any
            return 'exception'
    else:
        return 'no website'

def get_pipedrive(get, params=None):
    url = f"{pipedriveURL}{get}{token}"             #Building the url
    response = requests.get(url, params=params)     #using request
    return response.json()

def put_pipedrive(put, params=None):
    url = f"{pipedriveURL}{put}{token}"             #Building the url
    response = requests.put(url, data=params)     #using request
    return response.json()

In [43]:
#------- Creating Tables
#Create Database
db_path = os.path.join("data", "pipedrive.db")
connection = sqlite3.connect(db_path)

#Organization Table
CREATE_ORG_TABLE = """ 
CREATE TABLE IF NOt EXISTS Organization (
    org_id INTEGER PRIMARY KEY,
    name TEXT,
    website TEXT,
    page INTEGER
);"""
connection.execute(CREATE_ORG_TABLE)

#DEALS table
CREATE_DEAL_TABLE = """ 
CREATE TABLE IF NOT EXISTS DEALS (
   deal_id INTEGER PRIMARY KEY,
    org_id INTEGER,
    title TEXT,
    owner TEXT,
    value INTEGER,
    pipeline_id INTEGER,
    stage_order_nr INTEGER,
    add_time DATETIME,
    update_time DATETIME,
    label TEXT,
    status TEXT,
    lost_reason TEXT,
    page_number INTEGER
);"""

connection.execute(CREATE_DEAL_TABLE)

### 2 - Get Pipedrive Data throught API

In [44]:
#---- Getting all Organizations, 
#starting from the max value page
cursor = connection.cursor()
cursor.execute('SELECT MAX(page) FROM Organization')
max_page = cursor.fetchone()[0]
morepages = True
website_key = '4873360cacbe0b1dd893824bc11938dfd53ec655' #Insert your website_key if necessary, see more in https://pipedrive.readme.io/docs/core-api-concepts-about-pipedrive-api

#-- Getting all organization
while morepages == True:
    get = 'organizations'
    params = {"limit":"500", "start":[max_page]}
    org = get_pipedrive(get,params)
    if org['data'] is not None:
        morepages = org['additional_data']['pagination']['more_items_in_collection']
        with connection:
            for i in range(0,len(org['data'])):
                insert_sql = """ INSERT OR REPLACE INTO Organization (org_id,name,website,page) VALUES (?,?,?,?);"""
                connection.execute(insert_sql, (org['data'][i]['id'],org['data'][i]['name'], org['data'][i][website_key],max_page))

            max_page += 500
    else:
        morepages = False

In [45]:
#---- Getting All deals

#Parameters
number = 0 #Use it when want to start a new table, just remove the page_number variable to this one
morepages = True

#Getting Last page
cursor = connection.cursor()
cursor.execute('SELECT MAX(page_number) FROM DEALS')
page_number = cursor.fetchone()[0]

#--- 
while morepages == True:
    #Capturing recently Deals
    get = 'deals'
    params = {"limit":"500", "start":[page_number]}
    deal = get_pipedrive(get,params)
    if deal['data'] is not None:
        morepages = deal['additional_data']['pagination']['more_items_in_collection']
        with connection:
            for i in range(0,len(deal['data'])):
                #Check if org_id_value is not null
                org_id_value = deal['data'][i]['org_id']['value'] if deal['data'][i]['org_id'] and deal['data'][i]['org_id']['value'] is not None else 'no value'

                insert_sql = """
                    INSERT OR REPLACE INTO DEALS (
                        deal_id,
                        org_id,
                        title,
                        owner,
                        value,
                        pipeline_id,
                        stage_order_nr,
                        add_time,
                        update_time,
                        label,
                        status,
                        lost_reason,
                        page_number
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?);
                """

                # Parâmetros
                params = (
                    deal['data'][i]['id'],
                    org_id_value,
                    deal['data'][i]['title'],
                    deal['data'][i]['owner_name'],
                    deal['data'][i]['value'],
                    deal['data'][i]['pipeline_id'],
                    deal['data'][i]['stage_id'],
                    deal['data'][i]['add_time'],
                    deal['data'][i]['update_time'],
                    deal['data'][i]['label'],
                    deal['data'][i]['status'],
                    deal['data'][i]['lost_reason'],
                    deal['additional_data']['pagination']['start']
                )

                connection.execute(insert_sql,params)
        page_number += 500
    else:
        morepages = False


In [46]:
#------ Updating the Deals
#Here, numbe_page in database DEALS will be null to indicate that was updated

#Parameters
morepages = True
page_number = 0

#Get the last update timestamp
cursor = connection.cursor()
cursor.execute('SELECT MAX(update_time) FROM DEALS') #Getting the last time deal updated from SQL
timestamp_str = cursor.fetchone()[0] #The value will return a string
timestamp_pipedrive = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S') #Converting the string into date
timestamp_subtracted = timestamp_pipedrive - timedelta(hours=4) #Subtracting 4 hours to get in Brazil time zone
timestamp = timestamp_subtracted.strftime('%Y-%m-%d %H:%M:%S') #Converted to last time deal update in Brazil time zone

#--- execute the loop
while morepages == True:
    # Capturing recently Deals
    get = 'recents'
    params = {"since_timestamp":[timestamp],"limit":"500", "start":[page_number], "items":"deal"}
    recent_data = get_pipedrive(get, params)  # Renomeando a variável para evitar conflito com 'deal'
    if recent_data['data'] is not None:
        morepages = recent_data['additional_data']['pagination']['more_items_in_collection']
        with connection:
            for i in range(0, len(recent_data['data'])):
                # Check if org_id_value is not null
                org_id_value = recent_data['data'][i]['data']['org_id'] if recent_data['data'][i]['data']['org_id'] and recent_data['data'][i]['data']['org_id'] is not None else 'no value'

                insert_sql = """
                    INSERT OR REPLACE INTO DEALS (
                        deal_id,
                        org_id,
                        title,
                        owner,
                        value,
                        pipeline_id,
                        stage_order_nr,
                        add_time,
                        update_time,
                        label,
                        status,
                        lost_reason
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """

                # Parâmetros
                params = (
                    recent_data['data'][i]['data']['id'],
                    org_id_value,
                    recent_data['data'][i]['data']['title'],
                    recent_data['data'][i]['data']['owner_name'],
                    recent_data['data'][i]['data']['value'],
                    recent_data['data'][i]['data']['pipeline_id'],
                    recent_data['data'][i]['data']['stage_id'],
                    recent_data['data'][i]['data']['add_time'],
                    recent_data['data'][i]['data']['update_time'],
                    recent_data['data'][i]['data']['label'],
                    recent_data['data'][i]['data']['status'],
                    recent_data['data'][i]['data']['lost_reason']
                )

                connection.execute(insert_sql, params)
        page_number += 500
    else:
        morepages = False

### 3 - Generate DealValidation table and the Dataframe to validate domains

In [47]:
# DROP TABLE IF IT EXISTS
connection.execute("DROP TABLE IF EXISTS DEALVALIDATION;")

# CREATE NEW TABLE TO JOIN THE NEW INFORMATIONS FROM DEALS AND ORGANIZATIONS
connection.execute("""
    CREATE TABLE DEALVALIDATION AS
    SELECT
        rd.org_id,
        o.name,
        o.website,
        rd.deal_id,
        rd.title,
        rd.owner,
        rd.pipeline_id,
        rd.stage_order_nr,
        rd.add_time,
        rd.update_time,
        rd.status,
        rd.lost_reason
    FROM
        DEALS rd
    LEFT JOIN
        Organization o ON rd.org_id = o.org_id;
""")

In [48]:
#--- Filtering the table Dealvalidation for testing and query for dataframe
query = f""" 
SELECT *
FROM DEALVALIDATION
WHERE
    pipeline_id = 33
    AND stage_order_nr = 196
    AND add_time >= '{timestamp}'
    AND status = 'open';
    """

df = pd.read_sql_query(query, connection)

In [50]:
#Making a standard URL format
df['URL'] = df['website'].apply(make_https)

#--- Apply get_status function 

# Working with DataFrame df
chunk_size = 4  # Try different chunk sizes
chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Process the chunks in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda chunk: chunk.assign(validation=chunk['URL'].map(get_status)), chunks))

# Concatenate the results
df = pd.concat(results, ignore_index=True)

In [ ]:
#Get only validation, websit and deal_id column an then filtering just to not valids
df1 = df[['validation','deal_id','URL','title']]
df1 = df1[df1['validation'].isin(['valid', 'Other HTML']) == False]
df1.tail()
#df.to_excel("DealInvalid.xlsx", index = False) #to export

invalid_ids = df1['deal_id'].tolist()
df1.head()

### 4 - Apply lost to all invalid deals

In [52]:
########################## APPLYING LOST TO ALL INVALID DEALS
#--- Aplying lost api pipedrive to the deals that contains invalid URL

for invalid in invalid_ids:
    put = f'deals/{invalid}'
    params = {"status":'lost', "lost_reason": "Invalid Website" }
    put_pipedrive(put,params)